In [ ]:
import numpy as np
from scipy.integrate import quad

#Heston’s Stochastic Volatility Model

Heston’s Stochastic Volatility Model under real world probability measure:

$$\begin{aligned}
&d S_{t}=\mu S_{t} d t+\sqrt{v_{t}} S_{t} d W_{1, t}^{\mathbb{P}} \\
&d v_{t}=\kappa\left(\theta-v_{t}\right) d t+\sigma \sqrt{v_{t}} d W_{2, t}^{\mathbb{P}} \\
&\rho d t=d W_{2, t}^{\mathbb{P}} d W_{2, t}^{\mathbb{P}}
\end{aligned}$$

Using Girsanov's Thereom to $\mathbb{P} \rightarrow \mathbb{Q}$:

$$
\begin{aligned}
&d W_{S, t}^{\mathbb{Q}}=d W_{S, t}^{\mathbb{P}}+\alpha_{S} d t, \alpha_{S}=\frac{\mu_{\mathrm{p}}-r}{v_{t}} \\
&d W_{v, t}^{\mathbb{Q}}=d W_{v, t}^{\mathbb{P}}+\alpha_{v} d t, \alpha_{v}=\frac{\lambda}{\sigma^{\mathbb{P}}} \sqrt{v_{t}}
\end{aligned}
$$

We then obtain Heston’s Stochastic Volatility Model under risk-neutral measure:

$$
\begin{aligned}
&d S_{t}=r S_{t} d t+\sqrt{v_{t}} S_{t} d W_{1, t}^{\mathbb{Q}} \\
&d v_{t}=\kappa^{\mathbb{Q}}\left(\theta^{\mathbb{Q}}-v_{t}\right) d t+\sigma \sqrt{v_{t}} d W_{2, t}^{\mathbb{Q}} \\
&\rho^{\mathbb{Q}} d t=d W_{2, t}^{\mathbb{Q}} d W_{2, t}^{\mathbb{Q}}
\end{aligned}
$$

$$
\rho^{\mathbb{Q}}=\rho, \kappa^{\mathbb{Q}}=\kappa+\lambda, \theta^{\mathbb{Q}}=\kappa \theta /(\kappa+\lambda)
$$

Notations:

*   $S(t)$: Equity spot price, financial index
*   $V(t)$: Variance
* $C$: European call option price
* $K$: Strike price
* $W_{1,2}$: Standard Brownian movements
*$r$: Interest rate
*$q$: Dividend yield
*$\kappa $: Mean reversion rate
*$θ$: Long run variance
*$V_0$: Initial variance
*$σ$: Volatility of variance
*$ρ$: Correlation parameter
*$t_0$: Current date
*$T$: Maturity date
*$\lambda$: Variance risk premium


The paper [Heston’s Stochastic Volatility
Model Implementation,
Calibration and Some Extensions](https://www.maths.univ-evry.fr/pages_perso/crepey/Finance/051111_mikh%20heston.pdf) provided the following solution to Heston model. 

First, using standard arbitrage arguments we arrive at Garman’s partial differential equation:
$$
\frac{\delta C}{\delta t}+\frac{S^{2} v}{2} \frac{\delta^{2} C}{\delta S^{2}}+r S \frac{\delta C}{\delta S}-r C+[\kappa(\theta-v)-\lambda v] \frac{\delta C}{\delta v}+\frac{\sigma^{2} v}{2} \frac{\delta^{2} C}{\delta v^{2}}+\rho \sigma S v \frac{\delta^{2} C}{\delta S \delta v}=0
$$ 

Using method of characteristic functions, we bulit a solution to the above PDE in a similar form to the  Black-Scholes model:

$$
C\left(S_{0}, K, v_{0}, \tau\right)=S P_{1}-K e^{-r \tau} P_{2}
$$
where
- $P_{1}$ is the delta of the European call option and
- $P_{2}$ is the conditional risk neutral probability that the asset price will be greater than $\mathrm{K}$ at the maturit
$y .$

Given characteristic functions $\varphi_{1,2}$, both probabilities $P_{1,2}$ satisfy Garman’s PDE.  $P_{1,2}$ defined via the inverse Fourier transformation:

$$
\begin{aligned}
&X=\ln (S) \\
&P_{j}=\frac{1}{2}+\frac{1}{\pi} \int_{0}^{\inf } \Re\left[\frac{e^{-i \phi \ln K} \varphi_{j}\left(X_{0}, K, v_{0}, \tau ; \phi\right)}{i \phi}\right] d \phi, j \in\{1,2\}
\end{aligned}
$$

While the characteristic functions have the form:

$\varphi_{j}\left(X_{0}, K, v_{0}, \tau ; \phi\right)=e^{C(\tau ; \phi)+D(\tau ; \phi) v+i \phi X},\ j = 1,2$


With:

$\begin{aligned}
&\left.C(\tau ; \phi)=r \phi i \tau+\frac{a}{\sigma^{2}}\left[\left(b_{j}-\rho \sigma \phi i+d\right) \tau-2 \ln \left[\frac{1-g e^{d \tau}}{1-g}\right]\right)\right] \\
&D(\tau ; \phi)=\frac{b_{j}-\rho \sigma \phi i+d}{\sigma^{2}}\left[\frac{1-e^{d \tau}}{1-g e^{d \tau}}\right] \\
&-g=\frac{b_{j}-\rho \sigma \phi i+d}{b_{j}-\rho \sigma \phi i-d} \\
&-d=\sqrt{\left(\rho \sigma \phi i-b_{j}\right)^{2}-\sigma^{2}\left(2 u_{j} \phi i-\phi^{2}\right)} \\
&-u_{1}=0.5, u_{2}=-0.5-a=\kappa \theta \\
&-b_{1}=\kappa+\lambda-\rho \sigma \\
&-b_{2}=\kappa+\lambda
\end{aligned}$

## Implement the characteristic function

$\varphi\left(X_{0}, K, v_{0}, \tau ; \phi\right)=e^{r \phi i \tau} S^{i \phi}\left[\frac{1-g e^{d \tau}}{1-g}\right]^{\frac{2 a}{\sigma^{2}}} \exp \left[\frac{a \tau}{\sigma^{2}}\left(b_{2}-\rho \sigma \phi i+d\right)+\frac{v_{0}}{\sigma^{2}}\left(b_{2}-\rho \sigma \phi i+d\right)\left[\frac{1-e^{d \tau}}{1-g e^{d \tau}}\right]\right]$

where d and g no longer change with b1, b2 or u1, u2.


$\begin{aligned}
-\ d &=\sqrt{(\rho \sigma \phi i-b)^{2}+\sigma^{2}\left(\phi i+\phi^{2}\right)} \\
-\ g &=\frac{b-\rho \sigma \phi i+d}{b-\rho \sigma \phi i-d} \\
-\ a &=\kappa \theta \\
-\ b &=\kappa+\lambda
\end{aligned}$

In [ ]:
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
  """
  Returns the result of implementing the characteristic function.

  Args:
  - phi: variable in characteristic function
  - S0: float, initial spot price
  - v0: float, initial volatility
  - kappa: float, mean reversion rate
  - theta: float, long run variance
  - sigma: float, volatility of variance
  - rho: correlation parameter between price and volatility process
  - lambd: variance risk premium
  - tau: day to maturity
  - r: interest rate
  """
  a = kappa*theta
  b = kappa+lambd

  rspi = rho*sigma*phi*1j
  d = np.sqrt( (rspi - b)**2 + sigma**2*(phi*1j + phi**2) )
  g = (b - rspi + d)/(b - rspi - d)

  exp1 = np.exp(r*phi*1j*tau)
  term2 = S0**(phi*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
  exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)
  return exp1*term2*exp2

## Define the integrand as a function
$$
\int_{0}^{\inf } \Re\left[e^{r \tau} \frac{\varphi(\phi-i)}{i \phi K^{i \phi}}-K \frac{\varphi(\phi)}{i \phi K^{i \phi}}\right] d \phi
$$

In [ ]:
def integrand(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
  """
  Returns the result of implementing the integrand.

  Args:
  - phi: variable in characteristic function
  - S0: float, initial spot price
  - v0: float, initial volatility
  - kappa: float, mean reversion rate
  - theta: float, long run variance
  - sigma: float, volatility of variance
  - rho: correlation parameter between price and volatility process
  - lambd: variance risk premium
  - tau: day to maturity
  - r: interest rate
  """
  args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
  numerator = np.exp(r*tau) * heston_charfunc(phi-1j, *args) - K * heston_charfunc(phi, *args)
  denominator = 1j*phi*K**(1j*phi)
  return numerator/denominator


## Perform numerical integration over integrand and calculate option price.
$$
C\left(S_{0}, K, v_{0}, \tau\right)=\frac{1}{2}\left(S_{0}-K e^{-r \tau}\right)+\frac{1}{\pi} \int_{0}^{\inf } \Re\left[e^{r \tau} \frac{\varphi(\phi-i)}{i \phi K^{i \phi}}-K \frac{\varphi(\phi)}{i \phi K^{i \phi}}\right] d \phi
$$

In [ ]:
def heston_price(K, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
  """
  Return the option price calculated by heston model.

  Args:
  - phi: variable in characteristic function
  - S0: float, initial spot price
  - v0: float, initial volatility
  - kappa: float, mean reversion rate
  - theta: float, long run variance
  - sigma: float, volatility of variance
  - rho: correlation parameter between price and volatility process
  - lambd: variance risk premium
  - tau: day to maturity
  - r: interest rate
  """
  args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
  real_integral, err = np.real(quad(integrand, 0, 100, args = args))
  return (S0 - K*np.exp(-r*tau))/2 + real_integral/np.pi

##Test the model with dummy parameters.

In [ ]:
S0 = 100. # initial asset price
K = 100. # strike
v0 = 0.1 # initial variance
r = 0.03 # risk free rate
kappa = 1.5768 # rate of mean reversion of variance process
theta = 0.0398 # long-term mean variance
sigma = 0.3 # volatility of volatility
lambd = 0.575 # risk premium of variance
rho = -0.5711 # correlation between variance and stock process
tau = 1. # time to maturity

heston_price(K, S0, v0, kappa, theta, sigma, rho, lambd, tau, r)

11.540361819355377